Чтение файлов .seg

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Прочитаем текст из файла .seg, выведем его построчно на экран. Какая информация содержится в каждой строке? Какой формат записи?

In [ ]:
!wget https://pkholyavin.github.io/mastersprogramming/cta0001.seg_B2

--2024-10-08 09:48:57--  https://pkholyavin.github.io/mastersprogramming/cta0001.seg_B2
Resolving pkholyavin.github.io (pkholyavin.github.io)... 185.199.110.153, 185.199.108.153, 185.199.109.153, ...
Connecting to pkholyavin.github.io (pkholyavin.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 239 [application/octet-stream]
Saving to: ‘cta0001.seg_B2’

cta0001.seg_B2      100%[===================>]     239  --.-KB/s    in 0s      

2024-10-08 09:48:57 (4.00 MB/s) - ‘cta0001.seg_B2’ saved [239/239]



In [ ]:
with open("cta0001.seg_B2", "r", encoding="utf-8-sig") as f:
    lines = f.readlines()

for i, line in enumerate(lines):
    print(f"{i}\t{line.strip()}")

0	[PARAMETERS]
1	SAMPLING_FREQ=22050
2	BYTE_PER_SAMPLE=2
3	CODE=0
4	N_CHANNEL=1
5	N_LABEL=13
6	[LABELS]
7	0,32,j
8	8492,32,u0
9	12708,32,r'
10	13708,32,i4 j
11	18180,32,t
12	24904,32,r'
13	25940,32,i0
14	31544,32,f
15	36806,32,a4
16	38604,32,n
17	41618,32,a4
18	44508,32,f
19	54662,32,


Соответствие уровней:

In [ ]:
from itertools import product
letters = "GBRY"
nums = "1234"
levels = [ch + num for num, ch in product(nums, letters)]
print(levels)
level_codes = [2 ** i for i in range(len(levels))]
print(level_codes)

['G1', 'B1', 'R1', 'Y1', 'G2', 'B2', 'R2', 'Y2', 'G3', 'B3', 'R3', 'Y3', 'G4', 'B4', 'R4', 'Y4']
[1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096, 8192, 16384, 32768]


In [ ]:
level2code = {i: j for i, j in zip(levels, level_codes)}
code2level = {j: i for i, j in zip(levels, level_codes)}

In [ ]:
code2level

{1: 'G1',
 2: 'B1',
 4: 'R1',
 8: 'Y1',
 16: 'G2',
 32: 'B2',
 64: 'R2',
 128: 'Y2',
 256: 'G3',
 512: 'B3',
 1024: 'R3',
 2048: 'Y3',
 4096: 'G4',
 8192: 'B4',
 16384: 'R4',
 32768: 'Y4'}

Хак для определения кодировки:

In [ ]:
def detect_encoding(file_path):
    encoding = "utf-8"
    try:
        l = open(file_path, 'r', encoding="utf-8").read()
        if l.startswith("\ufeff"):  # т.н. byte order mark
            encoding = "utf-8-sig"
    except UnicodeDecodeError:
        try:
            open(file_path, 'r', encoding="utf-16").read()
            encoding = "utf-16"
        except UnicodeError:
            encoding = "cp1251"
    return encoding

(в общем случае это не работает, например:

In [ ]:
with open("lol.txt", "w", encoding="cp1251") as f:
    f.write("Пё")

In [ ]:
with open("lol.txt", "r", encoding="utf-8") as f:
    print(f.read())  # Greek Capital Letter Sho ???

ϸ


)

Задания для выполнения в классе:

1. Напишите функцию, которая принимает на вход имя файла .seg и возвращает словарь, который содержит всю информацию из секции ```[PARAMETERS]```. Не забудьте преобразовать данные в целочисленный тип!  
Пример словаря:

In [ ]:
{
    "SAMPLING_FREQ": 22050,
    "BYTE_PER_SAMPLE": 2,
    "CODE": 0,
    "N_CHANNEL": 1,
    "N_LABEL": 13
}

In [ ]:
def read_seg(filename: str):
  with open(filename, encoding=detect_encoding(filename)) as f:
    lines = [line.strip() for line in f.readlines()]

    params_start = lines.index("[PARAMETERS]") + 1
    labels_start = lines.index("[LABELS]") + 1

    params_lines = lines[params_start : labels_start - 1]
    params = {}
    for line in params_lines:
      key, value = line.split("=")
      params[key] = int(value)

    label_lines = lines[labels_start:]
    labels = []
    for line in label_lines:
      if line.count(",") < 2:
        break
      position, level, name = line.split(",")
      label = {
          "position": int(position) // params["BYTE_PER_SAMPLE"] // params["N_CHANNEL"],
          "level": code2level[int(level)],
          "name": name
      }
      labels.append(label)
  return params, labels

In [ ]:
params, labels = read_seg("/content/drive/MyDrive/cta/cta0001.seg_B1")
labels

[{'position': 0, 'level': 'B1', 'name': 'j'},
 {'position': 4246, 'level': 'B1', 'name': 'u0'},
 {'position': 6354, 'level': 'B1', 'name': "r'"},
 {'position': 6854, 'level': 'B1', 'name': 'i4'},
 {'position': 9090, 'level': 'B1', 'name': 't'},
 {'position': 12452, 'level': 'B1', 'name': "r'"},
 {'position': 12970, 'level': 'B1', 'name': 'i0'},
 {'position': 15772, 'level': 'B1', 'name': 'f'},
 {'position': 18403, 'level': 'B1', 'name': 'a4'},
 {'position': 19302, 'level': 'B1', 'name': 'n'},
 {'position': 20809, 'level': 'B1', 'name': 'a4'},
 {'position': 22254, 'level': 'B1', 'name': 'f'},
 {'position': 27331, 'level': 'B1', 'name': ''}]

2. Расширьте функцию так, чтобы она возвращала ещё и список меток. Сделайте каждую метку словарём с тремя полями: позиция **в отсчётах**, уровень **в текстовом представлении**, имя метки (строка).  
Как перевести из номера байта в номер отсчёта? На что нужно разделить?  
Пример списка меток:

In [ ]:
{"position": 0, "level": "B2", "name": "j"}

3. Выведите на экран все интервалы из файла .seg: попарно напечатайте первую и вторую метку, вторую и третью, третью и четвёртую, ..., предпоследнюю и последнюю.

In [ ]:
def print_label_pairs(filename):
  params, labels = read_seg(filename)
  for start, end in zip(labels, labels[1:]):
    print(start["name"], start["position"], end["position"])

In [ ]:
print_label_pairs("/content/drive/MyDrive/cta/cta0001.seg_B1")

j 0 4246
u0 4246 6354
r' 6354 6854
i4 6854 9090
t 9090 12452
r' 12452 12970
i0 12970 15772
f 15772 18403
a4 18403 19302
n 19302 20809
a4 20809 22254
f 22254 27331


4. Напишите функцию, которая принимает на вход словарь с параметрами и список меток и записывает файл .seg

In [ ]:
def write_seg(filename: str, params: dict, labels: list):
  params_defaults = {
    "SAMPLING_FREQ": 44100,
    "BYTE_PER_SAMPLE": 2,
    "CODE": 0,
    "N_CHANNEL": 1,
    "N_LABEL": 0
  }
  params_defaults.update(params)
  params_defaults["N_LABEL"] = len(labels)
  with open(filename, "w", encoding="utf-8") as f:
    f.write("[PARAMETERS]\n")
    for key in params_defaults.keys():
      f.write(f"{key}={params_defaults[key]}\n")
    f.write("[LABELS]\n")
    for label in labels:
      f.write(str(label["position"] * params_defaults["BYTE_PER_SAMPLE"] * params_defaults["N_CHANNEL"]))
      f.write(",")
      f.write(str(level2code[label["level"]]))
      f.write(",")
      f.write(label["name"])
      f.write("\n")

In [ ]:
params = {
    "SAMPLING_FREQ": 22050,
    "BYTE_PER_SAMPLE": 2,
    "CODE": 0,
    "N_CHANNEL": 1,
    "N_LABEL": 0
  }
labels = [{"position": 0, "level": "G1", "name": "1243"}, {"position": 100, "level": "G1", "name": "7876"}]
write_seg("12345.seg", params, labels)

5. Напишите функцию, которая принимает на вход два имени файлов ```.seg```, рассматривает каждую пару меток в первом файле, регистрирует, какие метки из второго файла находятся между ними и записывает текстовый файл в следующем формате:
```
<время метки1_1 в секундах> <время метки1_2 в секундах> <имя метки1_1> <имя метки2_1> <имя метки2_2> ...
```

Например:
```
0.0 0.412 юрий j u0 r' i4
0.412 1.24 трифонов t r' i0 f a4 n a4 f
```

In [ ]:
def make_textfile_with_labels(fn1: str, fn2: str, filename):
  params1, labels1 = read_seg(fn1)
  params2, labels2 = read_seg(fn2)
  with open(filename, "w", encoding="utf-8") as f:
    for start, end in zip(labels1, labels1[1:]):
      new_string = ""
      for label in labels2:
        if start["position"] <= label["position"] < end["position"]:
          new_string += label["name"] + " "
      st_time = round(start["position"] / params1["SAMPLING_FREQ"], 3)
      end_time = round(end["position"] / params1["SAMPLING_FREQ"], 3)
      f.write(f"{st_time} {end_time} {start['name']} {new_string}\n")

In [ ]:
make_textfile_with_labels("/content/drive/MyDrive/cta/cta0001.seg_Y1", "/content/drive/MyDrive/cta/cta0001.seg_B1", "new_text_file.txt")

Домашнее задание: напишите программу, которая обрабатывает файл .seg_B1 (с границами звуков) и генерирует параллельный ему файл .seg_Y3, содержащий границы открытых слогов (по Бондарко)

Должно получиться так:

<img src="https://pkholyavin.github.io/year4programming/syllables_result_example.png" width="1000">